In [6]:
from copy import deepcopy

import imageio
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn
from torchvision import models, transforms
from PIL import Image

In [7]:
transforms = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])

In [8]:
activation = {}

def get_activation(name):
    def hook(model, input, output):        
        activation[name] = output.detach()
    return hook

In [9]:
def get_cam(input_image, model, transforms):
    # transform input image
    #image = Image.open(input_image).convert('RGB')
    input_data = transforms(input_image).unsqueeze(0)
    
 
    # attach hook to layer 4
    model.layer4.register_forward_hook(get_activation("layer4"))
    
    model.eval();
    with torch.no_grad():
        preds = model(input_data)
        preds_softmax = torch.nn.functional.softmax(preds, dim=1)
        top_prob, top_pred = preds_softmax.max(dim=1)

        return top_pred, top_prob, activation
   

    pred_class, pred_prob, activation = get_data(image, model, transforms)

    fc_weights = model.fc[1].weight[pred_class, :].unsqueeze(2).unsqueeze(3)
    fc_weights.shape

    cam=torch.einsum("bchw,bchw->bhw", fc_weights, activation["layer4"])

    # add dimension to make compatible with interpolation
    cam = cam.unsqueeze(0)
    cam.shape

    final_cam = F.interpolate(cam, tuple(image.size[:2]), mode="bilinear")


    plt.imshow(image)
    plt.imshow(final_cam.squeeze().detach().numpy(), alpha=0.6)

    plt.show()

In [10]:
input_image = Image.open("/mnt/d/Lung_progression/data_10x/test/prog/S1_tile_187_level1_8114-18002-8626-18514.png").convert('RGB')

In [14]:
#model = TheModelClass(*args, **kwargs)
model = torch.load('/mnt/d/Lung_progression/model')


In [15]:
model

OrderedDict([('conv1.weight',
              tensor([[[[-7.0170e-03, -2.6764e-03,  1.6900e-03,  ...,  5.8850e-02,
                          1.9586e-02, -1.0552e-02],
                        [ 1.3543e-02,  1.2608e-02, -1.0752e-01,  ..., -2.7001e-01,
                         -1.2677e-01,  5.9079e-03],
                        [-2.6764e-03,  6.1931e-02,  2.9636e-01,  ...,  5.1812e-01,
                          2.5831e-01,  6.7157e-02],
                        ...,
                        [-2.5438e-02,  1.6705e-02,  7.2770e-02,  ..., -3.3308e-01,
                         -4.1857e-01, -2.5346e-01],
                        [ 3.3297e-02,  4.2567e-02,  6.3725e-02,  ...,  4.1294e-01,
                          3.9510e-01,  1.6788e-01],
                        [-1.0967e-02, -3.1838e-03, -2.2151e-02,  ..., -1.4960e-01,
                         -8.1518e-02, -4.8887e-03]],
              
                       [[-9.8626e-03, -2.3988e-02, -3.1608e-02,  ...,  3.6309e-02,
                          4.7977